<a target="_blank" href="https://colab.research.google.com/github/wbfrench1/barker_DATA606/blob/main/src/draft_proposal_eda.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# EDA - Takeaways

<ol><li>There is enough data.  Roughly 22,000 labeled movie trivia questions over two sets that are broken into training and test sets.</li>
<li>Entities are as expected, including ['ACTOR', 'YEAR','TITLE', 'GENRE' 'DIRECTOR', 'PLOT', 'CHARACTER', 'RATING', 'RATINGS_AVERAGE', 'TRAILER']</li>
<li>Data must be cleaned (e.g. CHARACTER entity includes "adam sandler" and "adam sandlers"

In [ ]:
import requests
import pandas as pd
import random
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [ ]:
def fetch_data( str_url: str, str_file_name_out: str):
    '''

    Description:    takes a string form to the url location of a dataset 
                    takes a string form of the filename to write the dataset to
                    saves the file to the working directory under the filename

    parameters:     str_url: a string containing the url of the data
                    str_file_name: a string containing the name the data will be
                       saved under in the current drive
    
    return:         Nothing
    ''' 
    r = requests.get(str_url, allow_redirects=True)
    open(str_file_name_out, 'wb').write(r.content)
    



def label_data_w_item_number ( str_f_name: str) -> pd.DataFrame:
    '''

    Description:    tags the same label to each word in a question to facilitate
                       identification of unique questions later
                    saves the updated file
    
    Parameters:     str_f_name: takes the filename of one of the files in the 
                       trivia dataset and appends a question number to each word
                       that is part of the same question.
    
    return:         None

    '''

    with open(str_f_name) as f:
        lines = f.readlines()

    i = 1
    with open(str_f_name, 'w') as f_out:
        for line in lines:
            if line != '\n':
                # add a label for each question
                line = str(i) + '\t' + line
                f_out.write(line)
            else:
                i += 1
    



def separate_label_into_pos_and_entity_components(df: pd.DataFrame
                                                  ) -> pd.DataFrame:
    '''

    Description:    takes the label column which contains two variables and 
                        separates them into their two component pieces 
                        pos and entity
    
    Parameters:     df: takes a dataframe of labeled movie questions
    
    return:         dataframe with two new columns, pos and entity

    '''

    # adjust the O label to include a separate entity called other
    df.loc[df['label'] == 'O', 'label' ] = 'O-OTHER'
    # put the entity location of the word in new column 'pos' for position
    df['pos'] = [label[0] for label in df.label.str.split('-')]
    # put the entity type in a new column called 'entity'
    df['entity'] = [label[1] for label in df.label.str.split('-')]

    return df




def merge_entity_components_into_one (df: pd.DataFrame,
                                      l_entity_types : list) -> pd.DataFrame:
    '''

    Description:    takes a dataframe of labeled movie questions where the
                       the entity type (e.g. character, genre) has been 
                       separated from the position (B-beginning, I-intermediate)
                       and reassembles the entity into a single unit.
    
    Parameters:     df: takes a dataframe of labeled movie questions which has
                       an entity, quest_num, and word field
    
    return:         dataframe with two columns, entity and word, where word 
                       represents a single entity where B,I parts are 
                       reassembled

    '''


    df_label = pd.DataFrame()
    # loop through each entity type
    for entity in l_entity_types:
        # get the unique indicies associated with each entity
        for index in df.loc[df['entity'] == entity, 'quest_num'].unique():

            word = ' '.join([str(word) for word in df.loc[(df['quest_num'] == index)
                                    & (df['entity'] == entity), 'word'].tolist()])
            df_label2 = pd.DataFrame( { 'entity': entity, 'word' : word},index=[index])
            df_label = df_label.append([df_label2])
    
    return df_label


# Data

## 1. Data Locations and Filenames

In [ ]:
str_url_1_train = 'https://groups.csail.mit.edu/sls/downloads/movie/engtrain.bio'
str_file_name_1_train = 'eng_train.bio'

str_url_1_test = 'https://groups.csail.mit.edu/sls/downloads/movie/engtest.bio'
str_file_name_1_test = 'eng_test.bio'

str_url_2_train = 'https://groups.csail.mit.edu/sls/downloads/movie/trivia10k13train.bio'
str_file_name_2_train = 'trivia_train.bio'

str_url_2_test = 'https://groups.csail.mit.edu/sls/downloads/movie/trivia10k13test.bio'
str_file_name_2_test = 'trivia_test.bio'

l_data_urls_and_filenames = [(str_url_1_train, str_file_name_1_train),
                            (str_url_1_test, str_file_name_1_test),
                            (str_url_2_train, str_file_name_2_train),
                            (str_url_2_test, str_file_name_2_test)]

## 2. Get Data and Save to Local Drive
## 3. Label Data with Question Numbers

In [ ]:
for str_url, str_f_name_out in l_data_urls_and_filenames:
    # 3. get data and save
    fetch_data( str_url = str_url,
            str_file_name_out = str_f_name_out)
    # label data with question numbers
    label_data_w_item_number( str_f_name = str_f_name_out)

In [ ]:
df = pd.read_csv('eng_train.bio', sep='\t', names=['quest_num', 'label', 'word'])

#### Show an example that each question is now numbered

In [ ]:
df.loc[df['quest_num'] == random.randint(0,df['quest_num'].max()+1)]

,quest_num,label,word
76498,7584,O,what
76499,7584,O,are
76500,7584,O,some
76501,7584,B-RATING,g
76502,7584,O,rated
76503,7584,B-RATINGS_AVERAGE,watchable
76504,7584,B-GENRE,sci
76505,7584,I-GENRE,fi
76506,7584,O,movies


In [ ]:
df.label.unique()

array(['O', 'B-ACTOR', 'I-ACTOR', 'B-YEAR', 'B-TITLE', 'B-GENRE',
       'I-GENRE', 'B-DIRECTOR', 'I-DIRECTOR', 'B-SONG', 'I-SONG',
       'B-PLOT', 'I-PLOT', 'B-REVIEW', 'B-CHARACTER', 'I-CHARACTER',
       'B-RATING', 'B-RATINGS_AVERAGE', 'I-RATINGS_AVERAGE', 'I-TITLE',
       'I-RATING', 'B-TRAILER', 'I-TRAILER', 'I-REVIEW', 'I-YEAR'],
      dtype=object)

B = Begin
I = Intermediate
E = End

In [ ]:
df.label.value_counts()

O                    61008
B-GENRE               4354
I-TITLE               3495
I-ACTOR               3474
B-ACTOR               3220
B-YEAR                2858
I-YEAR                2456
B-TITLE               2376
B-RATING              2007
B-PLOT                1927
B-RATINGS_AVERAGE     1869
I-DIRECTOR            1850
B-DIRECTOR            1720
I-PLOT                1687
I-RATINGS_AVERAGE     1673
I-RATING               840
I-GENRE                786
I-SONG                 446
B-CHARACTER            385
I-CHARACTER            342
B-SONG                 245
B-REVIEW               221
I-REVIEW               132
B-TRAILER              113
I-TRAILER                7
Name: label, dtype: int64

## 4. Separate the label field into two data elements: loc and pos

In [ ]:
df = separate_label_into_pos_and_entity_components(df)
df.sample(10)

,quest_num,label,word,pos,entity
80096,7914,I-TITLE,helper,I,TITLE
16430,2004,O-OTHER,a,O,OTHER
91211,8832,I-RATINGS_AVERAGE,liked,I,RATINGS_AVERAGE
68012,6893,O-OTHER,that,O,OTHER
96180,9351,I-YEAR,seven,I,YEAR
66660,6771,O-OTHER,the,O,OTHER
48065,5227,O-OTHER,in,O,OTHER
9473,1153,I-PLOT,show,I,PLOT
19272,2352,O-OTHER,in,O,OTHER
60403,6191,O-OTHER,there,O,OTHER


In [ ]:
df.loc[(df['quest_num'] == 2440)]

,quest_num,label,word,pos,entity
20006,2440,O-OTHER,find,O,OTHER
20007,2440,O-OTHER,me,O,OTHER
20008,2440,O-OTHER,the,O,OTHER
20009,2440,O-OTHER,movie,O,OTHER
20010,2440,O-OTHER,title,O,OTHER
20011,2440,O-OTHER,of,O,OTHER
20012,2440,O-OTHER,the,O,OTHER
20013,2440,O-OTHER,first,O,OTHER
20014,2440,B-CHARACTER,james,B,CHARACTER
20015,2440,I-CHARACTER,bond,I,CHARACTER


## 5. Identify the total number of questions - (9775)

In [ ]:
df['quest_num'].max()

9775

## 6. Create Single Entities from Component Words

BIO breaks entities up into beginning and intermediate pieces.  The below reassemblies the pieces into one entity.  Purpose is to understand the scope of the data set.

In [ ]:
' '.join(df.loc[(df['quest_num'] == 2440)
        & (df['label'] == 'CHARACTER'), 'word'].tolist())

''

In [ ]:
df.loc[df['label'] == 'CHARACTER', 'quest_num'].unique()

array([], dtype=int64)

In [ ]:
l_entities = list(df['entity'].unique())
l_entities.remove('OTHER')
l_entities

['ACTOR',
 'YEAR',
 'TITLE',
 'GENRE',
 'DIRECTOR',
 'SONG',
 'PLOT',
 'REVIEW',
 'CHARACTER',
 'RATING',
 'RATINGS_AVERAGE',
 'TRAILER']

In [ ]:
df_label = merge_entity_components_into_one(df,l_entities )

In [ ]:
df_label.entity.value_counts()

GENRE              4341
ACTOR              3080
YEAR               2854
TITLE              2371
RATING             2000
PLOT               1896
RATINGS_AVERAGE    1868
DIRECTOR           1711
CHARACTER           361
SONG                235
REVIEW              218
TRAILER             113
Name: entity, dtype: int64

In [ ]:
for entity in l_entities:
    print((df_label[df_label['entity'] == entity].value_counts()).sort_index()[:20], '\n\n')

entity  word                   
ACTOR   2pac shakur                 1
        a j langer                  1
        aaliyah                     1
        aaron kwok                  2
        acted                       1
        actor                       4
        actors                      1
        adam sandler               16
        adam sandlers               1
        adam storke                 1
        adrian pasdar               1
        african american actor      1
        aidan quinn                 4
        al lewis                    1
        al pacimp robert deniro     1
        al pacino                  14
        al pacino robert deniro     1
        al pacinos                  1
        alan alda                   2
        alan aldas                  1
dtype: int64 


entity  word     
YEAR    1800s          1
        1900s          1
        1920s          1
        1930           2
        1930s          5
        1933           1
        1939           1

# Repeat the above steps with each of the remaining three data frames

In [ ]:
dict_dfs = {}
dict_label_dfs = {}

for url, f_name in l_data_urls_and_filenames:
    print(f_name)
    dict_dfs[('df_'+ f_name.split('.')[0])] = pd.read_csv(f_name, sep='\t', names=['quest_num', 'label', 'word'])
    dict_dfs[('df_'+ f_name.split('.')[0])] = separate_label_into_pos_and_entity_components(dict_dfs[('df_'+ f_name.split('.')[0])])
    print('\n','Number of Questions: ', dict_dfs[('df_'+ f_name.split('.')[0])]['quest_num'].max(),'\n', sep='')
    l_entities = list(dict_dfs[('df_'+ f_name.split('.')[0])]['entity'].unique())
    l_entities.remove('OTHER')
    dict_label_dfs[('df_label_'+ f_name.split('.')[0])]= merge_entity_components_into_one(dict_dfs[('df_'+ f_name.split('.')[0])],l_entities )
    print('\n', dict_label_dfs[('df_label_'+ f_name.split('.')[0])].entity.value_counts(), '\n\n\n', sep='')

eng_test.bio

Number of Questions: 2443


GENRE              1115
ACTOR               763
YEAR                719
TITLE               562
RATING              498
PLOT                487
DIRECTOR            456
RATINGS_AVERAGE     449
CHARACTER            85
REVIEW               54
SONG                 51
TRAILER              30
Name: entity, dtype: int64



trivia_train.bio

Number of Questions: 7816


Plot              5980
Actor             3627
Genre             3297
Year              2688
Director          1742
Character_Name     812
Opinion            766
Origin             731
Relationship       524
Award              261
Quote              120
Soundtrack          46
Name: entity, dtype: int64



trivia_test.bio

Number of Questions: 1953


Plot              1459
Actor              944
Genre              780
Year               655
Director           415
Character_Name     225
Opinion            190
Origin             178
Relationship       147
Award               54
Quote        

In [ ]:
df.shape

(99491, 5)

In [ ]:
dict_label_dfs.keys()

dict_keys(['df_label_eng_test', 'df_label_trivia_train'])

In [ ]:
dict_label_dfs['df_label_trivia_train']

""


In [ ]:
dict_dfs['df_eng_test'].head()

,quest_num,label,word,pos,entity
0,1,O-OTHER,are,O,OTHER
1,1,O-OTHER,there,O,OTHER
2,1,O-OTHER,any,O,OTHER
3,1,O-OTHER,good,O,OTHER
4,1,B-GENRE,romantic,B,GENRE


In [ ]:
dict_dfs['df_trivia_train']

,quest_num,label,word,pos,entity
0,1,B-Actor,steve,B,Actor
1,1,I-Actor,mcqueen,I,Actor
2,1,O-OTHER,provided,O,OTHER
3,1,O-OTHER,a,O,OTHER
4,1,B-Plot,thrilling,B,Plot
...,...,...,...,...,...
158818,7816,I-Origin,on,I,Origin
158819,7816,I-Origin,a,I,Origin
158820,7816,I-Origin,nicholas,I,Origin
158821,7816,I-Origin,sparks,I,Origin


In [ ]:
(dict_label_dfs[('df_label_eng_test')].entity.value_counts())

In [ ]:
pd.read_csv('trivia_train.bio', sep='\t', names=['quest_num', 'label', 'word'])

,quest_num,label,word
0,1,B-Actor,steve
1,1,I-Actor,mcqueen
2,1,O,provided
3,1,O,a
4,1,B-Plot,thrilling
...,...,...,...
158818,7816,I-Origin,on
158819,7816,I-Origin,a
158820,7816,I-Origin,nicholas
158821,7816,I-Origin,sparks


- project business proposal

  - Literature Search
  - Run the Models
  - Show the results
  - Several Identify techniques for improvement
  - Implement techniques
  - Look at results
  - Appropriate Visualizatio of results
  
- eda -for each data set explain the data. 

  - Show the labels
  - Show the titles and counts